In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [10]:

# Define the column names for the DataFrame
column_names = ['Time', 'Query_time', 'CPU', 'Memory', 'Ping']

# Read the lines from the text file
with open('network/DDoS_A_dnsperf.txt', 'r') as file:
    lines = file.readlines()

# Initialize an empty list to store the extracted data
data = []

# Parse each line and extract the relevant information using regular expressions
for line in lines:
    match = re.match(r'(\d+:\d+:\d+); Query_time: (\d+); CPU: ([\d.]+)%%; Memory: (\d+)G?; Ping: ([\d.]+)', line)
    if match:
        data.append(match.groups())

# Create a DataFrame from the extracted data
df = pd.DataFrame(data, columns=column_names)
df['Query_time'] = df['Query_time'].astype(int)
df['Memory'] = df['Memory'].astype(int)
df['CPU'] = df['CPU'].astype(float)
df['Ping'] = df['Ping'].astype(float)
# Assuming 'df' is your DataFrame with the desired column named 'Time' as a datetime column
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.strftime('%H:%M:%S')

# Set the 'Time' column as the index of the DataFrame
df.set_index('Time', inplace=True)

# Display the DataFrame
print(df)


          Query_time   CPU Memory    Ping
Time                                     
15:43:05           8  7.98     14   3.745
15:43:07           7  7.25     14   2.237
15:43:09           5  5.30     14  10.470
15:43:11           6  4.22     14   3.943
15:43:13           9  5.50     14   2.728
...              ...   ...    ...     ...
15:46:58           4  5.51     14   5.097
15:47:00           3  5.20     14   2.977
15:47:02          23  5.40     14   2.896
15:47:04          10  6.60     14   3.079
15:47:06           5  4.90     14   6.521

[113 rows x 4 columns]


In [9]:
# Plot the desired column as a time series
df['CPU'].plot()

# Set plot title and labels
plt.title('Query Time over Time')
plt.xlabel('Time')
plt.ylabel('Query Time')

# Display the plot
plt.show()

TypeError: no numeric data to plot

Query_time      int32
CPU            object
Memory         object
Ping          float64
dtype: object
